This cell will load and define the initial data.

In [7]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Sample data
data = pd.DataFrame({
    'CPU': [2.5, 3.0, 2.8, 3.6, 2.2],
    'GPU': [4, 6, 4, 8, 4],
    'RAM': [8, 16, 8, 32, 8],
    'Storage': [256, 512, 256, 1024, 512],
    'Battery': [6, 4, 5, 6, 7],
    'Price': [700, 900, 750, 1500, 650],  # Price in dollars
    'Laptop': ['Laptop1', 'Laptop2', 'Laptop3', 'Laptop4', 'Laptop5']
})
data


,CPU,GPU,RAM,Storage,Battery,Price,Laptop
0,2.5,4,8,256,6,700,Laptop1
1,3.0,6,16,512,4,900,Laptop2
2,2.8,4,8,256,5,750,Laptop3
3,3.6,8,32,1024,6,1500,Laptop4
4,2.2,4,8,512,7,650,Laptop5


Data Preprocessing

In [8]:
# Step 1: Preprocess the data
features = ['CPU', 'GPU', 'RAM', 'Storage', 'Battery']
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[features])


Model Initialization

In [9]:
k = 3  # Adjustable 'k' value
knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
knn.fit(data_scaled)


NearestNeighbors(metric='euclidean', n_neighbors=3)

User Input

In [10]:
# Function to capture user preferences
def get_user_preferences():
    print("Enter your preferences for each feature:")
    cpu = float(input("CPU (e.g., 3.0 for 3.0 GHz): "))
    gpu = int(input("GPU (e.g., 6 for 6 GB): "))
    ram = int(input("RAM (e.g., 16 for 16 GB): "))
    storage = int(input("Storage (e.g., 512 for 512 GB): "))
    battery = int(input("Battery (e.g., 6 for 6 hours): "))
    min_price = float(input("Minimum Price (e.g., 600): "))
    max_price = float(input("Maximum Price (e.g., 1000): "))
    return [[cpu, gpu, ram, storage, battery]], min_price, max_price



Recommendation Logic

In [11]:
# Main recommendation logic
def recommend_laptops(user_input):
    # Transform user input with the same scaler
    user_input_scaled = scaler.transform(user_input)
    # Find nearest neighbors
    distances, indices = knn.kneighbors(user_input_scaled)
    # Retrieve recommendations
    recommended_laptops = data.iloc[indices[0]].copy()
    recommended_laptops['Distance'] = distances[0]
    # Filter based on price range
    recommended_laptops = recommended_laptops[
        (recommended_laptops['Price'] >= min_price) & (recommended_laptops['Price'] <= max_price)
    ]
    return recommended_laptops


Execution

In [12]:
# Capture user input in a separate session
user_input, min_price, max_price = get_user_preferences()

# Get and display recommendations
recommended_laptops = recommend_laptops(user_input)
print("\nRecommended Laptops within your price range:\n", recommended_laptops[['Laptop', 'CPU', 'GPU', 'RAM', 'Storage', 'Battery', 'Price', 'Distance']])


Enter your preferences for each feature:


ValueError: could not convert string to float: ''